In [ ]:
! pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import AdamW
import numpy as np
import pandas as pd
from datasets import load_dataset, concatenate_datasets
from transformers import AutoModel, AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Datasets

In [ ]:
def get_senteval_dataset(dataset_name):
    dataset = load_dataset(f'rahulsikder223/SentEval-{dataset_name}')
    concatenated_dataset = concatenate_datasets([dataset['train'], dataset['test']])
    concatenated_dataset = concatenated_dataset.rename_column("label", "labels")
    return concatenated_dataset

senteval_datasets = ['CR', 'MPQA', 'MR', 'SUBJ']

# Model

In [ ]:
model_id = 'bert-base-uncased'

# Loss Functions

## Classification Losses - Single sentence column and sentiment label

### Cross-Entropy Loss

In [ ]:
def cross_entropy_loss(logits, labels):
    return nn.CrossEntropyLoss()(logits, labels)

### Label Smoothing Cross-Entropy Loss

In [ ]:
def label_smoothing_cross_entropy_loss(logits, labels, smoothing=0.1):
    confidence = 1.0 - smoothing
    log_probs = F.log_softmax(logits, dim=-1)

    # Initializing true distribution with smoothing value for all classes...
    true_dist = torch.full_like(log_probs, smoothing / (log_probs.size(1) - 1))
    # Setting the true label confidence in the correct class...
    true_dist.scatter_(1, labels.unsqueeze(1), confidence)

    loss = torch.mean(torch.sum(-true_dist * log_probs, dim=-1))
    return loss

## Embedding Losses

### Triplets - Anchor, Positive and Negative sentences (3 sentence columns) and label

#### Triplet Loss

In [ ]:
def generate_triplets(embeddings, labels):
    anchors, positives, negatives = [], [], []
    for i, anchor_label in enumerate(labels):
        anchor = embeddings[i]

        # Finding a positive example (same label as anchor)...
        positive_indices = (labels == anchor_label).nonzero(as_tuple=True)[0].tolist()
        # Ensuring positive is not the same as anchor...
        positive_indices.remove(i)

        # Ensuring there is at least one valid positive example...
        if not positive_indices:
            continue

        # Selecting the first positive (no randomization for now)...
        positive_idx = positive_indices[0]
        positive = embeddings[positive_idx]

        # Finding a negative example (different label)...
        negative_indices = (labels != anchor_label).nonzero(as_tuple=True)[0].tolist()

        # Ensuring there is at least one valid negative example...
        if not negative_indices:
            continue

        # Select the first negative
        negative_idx = negative_indices[0]
        negative = embeddings[negative_idx]

        # Adding the triplet...
        anchors.append(anchor)
        positives.append(positive)
        negatives.append(negative)

    # Ensuring non-empty lists before stacking...
    if anchors and positives and negatives:
        return torch.stack(anchors), torch.stack(positives), torch.stack(negatives)
    else:
        return None, None, None

In [ ]:
def generate_all_triplets(embeddings, labels):
    anchors, positives, negatives = [], [], []
    for i, anchor_label in enumerate(labels):
        anchor = embeddings[i]

        # Find all positive examples (same label as anchor)
        positive_indices = (labels == anchor_label).nonzero(as_tuple=True)[0].tolist()
        positive_indices.remove(i)  # Exclude the anchor itself

        # Find all negative examples (different label from anchor)
        negative_indices = (labels != anchor_label).nonzero(as_tuple=True)[0].tolist()

        # Generate all valid triplets for this anchor
        for pos_idx in positive_indices:
            positive = embeddings[pos_idx]
            for neg_idx in negative_indices:
                negative = embeddings[neg_idx]

                # Add the triplet
                anchors.append(anchor)
                positives.append(positive)
                negatives.append(negative)

    # Convert lists to tensors for batch processing
    if anchors and positives and negatives:
        return torch.stack(anchors), torch.stack(positives), torch.stack(negatives)
    else:
        return None, None, None

In [ ]:
def triplet_loss(embeddings, labels, margin=1.0):
    anchors, positives, negatives = generate_triplets(embeddings, labels)
    if anchors is None or positives is None or negatives is None:
        return 0.0

    # Euclidean distance between anchor and positive, and anchor and negative...
    pos_dist = F.pairwise_distance(anchors, positives)
    neg_dist = F.pairwise_distance(anchors, negatives)
    loss = torch.clamp(pos_dist - neg_dist + margin, min=0.0)
    return loss.mean()

In [ ]:
def triplet_all_loss(embeddings, labels, margin=1.0):
    anchors, positives, negatives = generate_all_triplets(embeddings, labels)
    if anchors is None or positives is None or negatives is None:
        return 0.0

    # Euclidean distance between anchor and positive, and anchor and negative...
    pos_dist = F.pairwise_distance(anchors, positives)
    neg_dist = F.pairwise_distance(anchors, negatives)
    loss = torch.clamp(pos_dist - neg_dist + margin, min=0.0)
    return loss.mean()

#### Hard Triplet Loss

In [ ]:
def _pairwise_distances(embeddings, squared=False):
    """Computes the 2D matrix of distances between all the embeddings.
    """
    dot_product = torch.matmul(embeddings, embeddings.t())
    square_norm = torch.diag(dot_product)

    # Computing the pairwise distance matrix as we have:
    # ||a - b||^2 = ||a||^2  - 2 <a, b> + ||b||^2
    distances = square_norm.unsqueeze(0) - 2.0 * dot_product + square_norm.unsqueeze(1)
    distances[distances < 0] = 0

    if not squared:
        mask = distances.eq(0).float()
        distances = distances + mask * 1e-16
        distances = (1.0 -mask) * torch.sqrt(distances)
    return distances

def _get_anchor_positive_triplet_mask(labels):
    """Returns a 2D mask where mask[a, p] is True iff a and p are distinct and have same label.
    """
    # Checking that i and j are distinct...
    indices_equal = torch.eye(labels.size(0), device=labels.device).bool()
    indices_not_equal = ~indices_equal

    # Checking if labels[i] == labels[j]...
    labels_equal = labels.unsqueeze(0) == labels.unsqueeze(1)

    return labels_equal & indices_not_equal

def _get_anchor_negative_triplet_mask(labels):
    """Returns a 2D mask where mask[a, n] is True iff a and n have distinct labels.
    """
    # Check if labels[i] != labels[k]
    return ~(labels.unsqueeze(0) == labels.unsqueeze(1))

In [ ]:
def hard_triplet_loss(embeddings, labels, margin=1.0):
    # Calculating pairwise distance matrix - SBERT
    pairwise_dist = _pairwise_distances(embeddings, squared=False)

    # Mask to get the hardest positive distances - SBERT
    mask_anchor_positive = _get_anchor_positive_triplet_mask(labels).float()
    anchor_positive_dist = mask_anchor_positive * pairwise_dist
    hardest_positive_dist, _ = anchor_positive_dist.max(dim=1, keepdim=True)

    # Mask to get the hardest negative distances - SBERT
    mask_anchor_negative = _get_anchor_negative_triplet_mask(labels).float()
    max_anchor_negative_dist, _ = pairwise_dist.max(dim=1, keepdim=True)
    anchor_negative_dist = pairwise_dist + max_anchor_negative_dist * (1.0 - mask_anchor_negative)
    hardest_negative_dist, _ = anchor_negative_dist.min(dim=1, keepdim=True)

    tl = hardest_positive_dist - hardest_negative_dist + margin
    # Ensuring non-negative loss
    tl = F.relu(tl)
    triplet_loss = tl.mean()

    return triplet_loss

### Pairs - 2 sentence columns and label

#### Pair Generation

In [ ]:
def generate_pairs(embeddings, labels):
    embedding1_list = []
    embedding2_list = []
    similarity_labels = []

    for i in range(len(labels)):
        for j in range(i + 1, len(labels)):
            embedding1 = embeddings[i]
            embedding2 = embeddings[j]

            # If the labels are the same, labeling the pair as 1 (similar)...
            if labels[i] == labels[j]:
                similarity_labels.append(1)
            else:
                # If the labels are different, labeling the pair as 0 (dissimilar)...
                similarity_labels.append(0)

            embedding1_list.append(embedding1)
            embedding2_list.append(embedding2)

    embedding1_tensor = torch.stack(embedding1_list)
    embedding2_tensor = torch.stack(embedding2_list)
    labels_tensor = torch.tensor(similarity_labels).to(labels.device)

    return embedding1_tensor, embedding2_tensor, labels_tensor

#### Cosine Similarity MSE

In [ ]:
def cosine_similarity_mse_loss(embeddings, labels):
    embedding1, embedding2, labels = generate_pairs(embeddings, labels)

    # Calculating the cosine similarity between the pairs of embeddings...
    cos_sim = F.cosine_similarity(embedding1, embedding2)

    # MSE loss...
    squared_difference = (labels - cos_sim) ** 2
    loss = squared_difference.mean()

    return loss

#### CoSENT Loss

In [ ]:
def cosent_loss(embeddings, labels, tau=20.0):
    embedding1, embedding2, labels = generate_pairs(embeddings, labels)

    # Input preparation...
    labels = (labels[:, None] < labels[None, :]).float()

    # Normalization of Logits...
    embedding1 = F.normalize(embedding1, p=2, dim=1)
    embedding2 = F.normalize(embedding2, p=2, dim=1)

    # Cosine Similarity Calculation...
    # The dot product of these pairs gives the cosine similarity, scaled by a factor of tau to control the sharpness of similarity scores...
    y_pred = torch.sum(embedding1 * embedding2, dim=1) * tau

    # Pairwise cosine similarity difference calculation...
    y_pred = y_pred[:, None] - y_pred[None, :]

    y_pred = (y_pred - (1 - labels) * 1e12).view(-1)

    zero = torch.Tensor([0]).to(y_pred.device)
    y_pred = torch.concat((zero, y_pred), dim=0)
    return torch.logsumexp(y_pred, dim=0)

#### In-Batch Negatives Loss

In [ ]:
def categorical_crossentropy(y_true, y_pred):
    return -(F.log_softmax(y_pred, dim=1) * y_true).sum(dim=1)

def in_batch_negative_loss(embeddings, labels, tau=20.0, negative_weights=0.0):
    device = labels.device
    embedding1, embedding2, labels = generate_pairs(embeddings, labels)

    y_pred = torch.empty((2 * embedding1.shape[0], embedding1.shape[1]), device=device)
    y_pred[0::2] = embedding1
    y_pred[1::2] = embedding2
    y_true = labels.repeat_interleave(2).unsqueeze(1)

    def make_target_matrix(y_true):
        idxs = torch.arange(0, y_pred.shape[0]).int().to(device)
        y_true = y_true.int()
        idxs_1 = idxs[None, :]
        idxs_2 = (idxs + 1 - idxs % 2 * 2)[:, None]

        idxs_1 *= y_true.T
        idxs_1 += (y_true.T == 0).int() * -2

        idxs_2 *= y_true
        idxs_2 += (y_true == 0).int() * -1

        y_true = (idxs_1 == idxs_2).float()
        return y_true

    neg_mask = make_target_matrix(y_true == 0)

    y_true = make_target_matrix(y_true)

    y_pred = F.normalize(y_pred, dim=1, p=2)
    similarities = y_pred @ y_pred.T
    similarities = similarities - torch.eye(y_pred.shape[0]).to(device) * 1e12
    similarities = similarities * tau

    if negative_weights > 0:
        similarities += neg_mask * negative_weights

    return categorical_crossentropy(y_true, similarities).mean()

#### Angle Loss

In [ ]:
def angle_loss(embeddings, labels, tau=1.0):
    embedding1, embedding2, labels = generate_pairs(embeddings, labels)

    # Input preparation...
    labels = (labels[:, None] < labels[None, :]).float()

    # Chunking into real and imaginary parts...
    y_pred_re1, y_pred_im1 = torch.chunk(embedding1, 2, dim=1)
    y_pred_re2, y_pred_im2 = torch.chunk(embedding2, 2, dim=1)

    a = y_pred_re1
    b = y_pred_im1
    c = y_pred_re2
    d = y_pred_im2

    z = torch.sum(c**2 + d**2, dim=1, keepdim=True)
    re = (a * c + b * d) / z
    im = (b * c - a * d) / z

    dz = torch.sum(a**2 + b**2, dim=1, keepdim=True)**0.5
    dw = torch.sum(c**2 + d**2, dim=1, keepdim=True)**0.5
    re /= (dz / dw)
    im /= (dz / dw)

    y_pred = torch.concat((re, im), dim=1)
    y_pred = torch.abs(torch.sum(y_pred, dim=1)) * tau
    y_pred = y_pred[:, None] - y_pred[None, :]
    y_pred = (y_pred - (1 - labels) * 1e12).view(-1)
    zero = torch.Tensor([0]).to(y_pred.device)
    y_pred = torch.concat((zero, y_pred), dim=0)
    return torch.logsumexp(y_pred, dim=0)

#### Combination of CoSENT, In-Batch Negatives and Angle Losses

In [ ]:
def cosent_ibn_angle(embeddings, labels, w_cosent=1, w_ibn=1, w_angle=1, tau_cosent=20.0, tau_ibn=20.0, tau_angle=1.0):
    return w_cosent * cosent_loss(embeddings, labels, tau_cosent) + w_ibn * in_batch_negative_loss(embeddings, labels, tau_ibn) + w_angle * angle_loss(embeddings, labels, tau_angle)

## Loss List

In [ ]:
losses = [
    {'loss_name': 'without_ft', 'loss_type': 'emb', 'loss_kwargs': {}},
    {'loss_name': 'cross_entropy_loss', 'loss_type': 'clf', 'loss_kwargs': {}},
    {'loss_name': 'label_smoothing_cross_entropy_loss', 'loss_type': 'clf', 'loss_kwargs': {'smoothing': 0.1}},
    {'loss_name': 'triplet_loss', 'loss_type': 'emb', 'loss_kwargs': {'margin': 5}},
    {'loss_name': 'triplet_all_loss', 'loss_type': 'emb', 'loss_kwargs': {'margin': 5}},
    {'loss_name': 'hard_triplet_loss', 'loss_type': 'emb', 'loss_kwargs': {'margin': 5}},
    {'loss_name': 'cosine_similarity_mse_loss', 'loss_type': 'emb', 'loss_kwargs': {}},
    {'loss_name': 'cosent_loss', 'loss_type': 'emb', 'loss_kwargs': {'tau': 20.0}},
    {'loss_name': 'in_batch_negative_loss', 'loss_type': 'emb', 'loss_kwargs': {'tau': 20.0}},
    {'loss_name': 'angle_loss', 'loss_type': 'emb', 'loss_kwargs': {'tau': 1.0}},
    {'loss_name': 'cosent_ibn_angle', 'loss_type': 'emb', 'loss_kwargs': {'w_cosent': 1, 'w_ibn': 1, 'w_angle': 1, 'tau_cosent': 20.0, 'tau_ibn': 20.0, 'tau_angle': 1.0}}
]

# Training

### Training Preparation (Model Preparation, Embedding Generation, Extraction, Training and Evaluation)

#### Device Setting

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#### Dataset Preparation

In [ ]:
def prepare_dataset(dataset, split=0.3):
    # Dataset Import...
    ds = get_senteval_dataset(dataset)

    # Random Split...
    train_test_split = ds.train_test_split(test_size=split)
    train_dataset = train_test_split['train']
    test_dataset = train_test_split['test']
    return train_dataset, test_dataset

#### Model and Tokenizer Preparation

In [ ]:
def get_model_tokenizer(model_id, loss_type='clf'):
    if loss_type == 'clf':
        model = AutoModelForSequenceClassification.from_pretrained(model_id)
    else:
        model = AutoModel.from_pretrained(model_id)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model.to(device)
    return model, tokenizer

#### Dataset Tokenization and Batch Processing

In [ ]:
def tokenize_dataset_batch(train_dataset, test_dataset, tokenizer, batch_size):
    def tokenize_function(examples):
        return tokenizer(examples['sentence'], padding='max_length', truncation=True, max_length=128)

    train_dataset = train_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Torch format setting...
    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

    # Batching using DataLoader...
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    return train_loader, test_loader

#### Embedding Extraction

In [ ]:
def extract_embeddings(model, device, dataloader):
    all_embeddings = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting embeddings", leave=False):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])

            # [CLS] token embeddings...
            embeddings = outputs.last_hidden_state[:, 0, :]
            all_embeddings.append(embeddings.cpu())
            all_labels.append(batch['labels'].cpu())

    return torch.cat(all_embeddings), torch.cat(all_labels)

#### Train Driver

In [ ]:
def train(model, train_loader, loss_type='clf', epochs=10, loss_name='cross_entropy_loss', **loss_kwargs):
    # Optimizer setting...
    optimizer = AdamW(model.parameters(), lr=5e-5)

    # Training loop...
    num_epochs = epochs
    model.train()
    for epoch in range(num_epochs):
        # print(f"Epoch {epoch + 1}/{num_epochs}")
        for batch in tqdm(train_loader, desc="Training", leave=False):
            batch = {k: v.to(device) for k, v in batch.items()}

            if loss_type == 'clf':
                # Cross-Entropy Losses...
                outputs = model(**batch)
                logits = outputs.logits
                loss = globals()[loss_name](logits, batch['labels'], **loss_kwargs)
            else:
                # Embedding Loss...
                outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])

                # [CLS] token embedding...
                embeddings = outputs.last_hidden_state[:, 0, :]
                loss = globals()[loss_name](embeddings, batch['labels'], **loss_kwargs)
                if loss == 0.0:
                    continue

            # Backpropagation...
            loss.backward()

            # Updating weights...
            optimizer.step()
            optimizer.zero_grad()
    return model

#### Evaluation Driver

In [ ]:
def evaluate_clf(model, test_loader):
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch in test_loader:
            batch = {k: v.to(device) for k, v in batch.items()}

            # Forward pass...
            outputs = model(**batch)
            predictions = outputs.logits.argmax(dim=-1)

            total_correct += (predictions == batch['labels']).sum().item()
            total_samples += batch['labels'].size(0)

    accuracy = total_correct / total_samples
    return accuracy

In [ ]:
def evaluate_emb(model, train_loader, test_loader):
    model.eval()

    # Generating embeddings of the train and test sentences...
    train_embeddings, train_labels = extract_embeddings(model, device, train_loader)
    test_embeddings, test_labels = extract_embeddings(model, device, test_loader)

    train_embeddings_np = train_embeddings.numpy()
    test_embeddings_np = test_embeddings.numpy()
    train_labels_np = train_labels.numpy()
    test_labels_np = test_labels.numpy()

    # Training a Logistic Regression classifier on the training embeddings...
    lr_clf = LogisticRegression(max_iter=10000)
    lr_clf.fit(train_embeddings_np, train_labels_np)

    # Predicting the labels for the test set...
    test_predictions = lr_clf.predict(test_embeddings_np)
    accuracy = accuracy_score(test_labels_np, test_predictions)
    return accuracy

### Loop

In [ ]:
total_runs = 3
batch_size = 60
accuracy_list = []
for loss in losses:
    loss_name = loss['loss_name']
    loss_type = loss['loss_type']
    loss_kwargs = loss['loss_kwargs']

    for dataset in senteval_datasets:
        print(f'Running: {loss_name} on {dataset}')
        total_accuracy = 0

        for loop_count in range(0, total_runs):
            # Dataset Preparation...
            train_dataset, test_dataset = prepare_dataset(dataset)

            # Model Preparation...
            model, tokenizer = get_model_tokenizer(model_id, loss_type)

            # Tokenize Batch...
            train_loader, test_loader = tokenize_dataset_batch(train_dataset, test_dataset, tokenizer, batch_size=batch_size)

            # Training Loop...
            if loss_name != 'without_ft':
                model = train(model, train_loader, loss_type, epochs=5, loss_name=loss_name, **loss_kwargs)

            # Evaluation loop...
            if loss_type == 'clf':
                accuracy = evaluate_clf(model, test_loader)
            else:
                accuracy = evaluate_emb(model, train_loader, test_loader)
            # print(f'Loop {loop_count} Accuracy - {accuracy}')
            total_accuracy += accuracy
        accuracy_list.append({'loss': loss_name, 'dataset': dataset, 'accuracy': total_accuracy / total_runs})